In [50]:
import pandas as pd
import numpy as np
import csv
import os
import xarray as xr
import plotly.express as px
# let's use the pyspa example for reference as we should be able to get some similar numbers

# intialize the trees; in prototype, they'll be lists (tuples?) of sectors,
# just pick one sector for initial construction
from typing import Optional, Self
from plotly import subplots

import plotly.graph_objects as go

# toy example
import copy
from warnings import warn
import numpy.typing as npt


In [111]:


def create_chain(transaction_table:npt.NDArray,final_demand:npt.NDArray,
                 direct_data:npt.NDArray,total_data:npt.NDArray,
                 sector_list:list[str],max_layers:int,
                 from_A:bool=False)->xr.Dataset:
    """Create a structure to house the data"""
    

    # todo coerce functions for column_vector, row_vector
    
    extended_table = np.concatenate([transaction_table,np.atleast_2d(final_demand).T],axis=1)
    output = np.sum(extended_table,axis=1)
    tech_coef = transaction_table/np.atleast_2d(output).T
    
    supply_chain_data = np.zeros((*tech_coef.shape,max_layers))
    layer_transfer = np.eye(tech_coef.shape[0])
    for layer in range(max_layers):
        # create the next level of dependency
        supply_chain_data[:,:,layer] = np.dot(layer_transfer,tech_coef)
        layer_transfer = copy.copy(supply_chain_data[:,:,layer])

    chain = xr.Dataset(data_vars={'transfer':(['supplier','user','layer'],
                                        supply_chain_data),
                                        'direct_intensity':(['supplier'],direct_data),
                                        'total_intensity':(['supplier'],total_data),
                                        'final_demand':(['supplier'],final_demand),
                                        'output':(['supplier'],output)}, 
                            coords={'supplier':sector_list,
                                    'user':sector_list,
                                    'layer':range(max_layers)})
    
    
    
    # construct other useful objects for downstream use
    direct = chain.output*chain.direct_intensity
    direct.name = 'absolute_direct'
    chain=chain.merge(direct)
    
    total = chain.output*chain.total_intensity
    total.name = 'absolute_total'
    chain=chain.merge(total)
    
    
    return chain



transaction_table = np.array([[0.5,.5,0,0.4],
                  [0,0.1,0.9,0],
                  [0.3,0.4,0.3,0.1],
                  [0,0,0.2,0.8],
                  ])

sector_list = ['atoms','bits','cats','dogs']
direct_data = [1.8,0.4,0.2,0.1]
total_data = [2.4,0.8,0.6,0.5]
final_demand = [1,2,0.8,1.2]
# might not be legit numbers but just checking math

max_layers = 10


toy_chain = create_chain(transaction_table,final_demand,direct_data,total_data,sector_list,max_layers)


In [143]:
# objects to construct a 'supply chain'
from uuid import uuid4

# class Process:
#     def __init__(self):
#         self.total_intensity = 0
#         self.direct_intensity = 0
#         self.tech_coef = 0
#         self.reliance = 0

class Process:
    def __init__(self,name):
        self.name = name
        self.id = uuid4()

class Node:
    def __init__(self,process_name:str,
                 children:Optional[list[Self]] = None):
        self.process_name = process_name
        self.children = children
        # values obtained from the SPA
        self.id = uuid4()
        # self.data = xr.Dataset # probably overkill, could just use an array tbh
        
        
    @property
    def terminal(self):
        return self.children is None
    
    def child_names(self):
        return " ".join([s.process for s in self.children])

    # def mass(self):
    #     if self.terminal:
    #         return 1
    #     else:
    #         return np.sum([c.mass() for c in self.children])+1
    def expand_name(self):
        if self.children is None:
            return [self.process_name]
        else:
            return [self.process_name,[x.expand_name() for x in self.children]]

    def add_child(self,new_node:Self):
        if self.children is None:
            self.children = [new_node]
        else:
            # can probably just append the ID
            self.children.append(new_node)
            # ensure this reference doesn't create circularity issues
            # new_node.add_parent(self.id)
            
    
root = Node(process_name='root')
ch1 = Node(process_name='ch1')
ch2 = Node(process_name='ch2')
ch3 = Node(process_name='ch3')
ch4= Node(process_name='ch4')

root.children = [ch1,ch2]
ch2.children = [ch3,ch4]
ch3.children = [ch4]

# assert root.mass() == 5
# assert not root.terminal 
# assert ch1.mass() == 1
# assert ch1.terminal 
# assert ch2.mass() == 3
# assert not ch2.terminal 
# assert ch3.mass() == 1
# assert ch3.terminal 

# expect paths:
# root, ch1
# root, ch2, ch3, ch4
# root, ch2, ch4


print(root.expand_name())

['root', [['ch1'], ['ch2', [['ch3', [['ch4']]], ['ch4']]]]]


In [156]:
from uuid import UUID



def find_links(weight_matrix,node_names,cutoff):
    new_nodes = []
    new_links = []
    
    for node in node_names:
        incoming_weights = weight_matrix.sel(user=node)    
        mask = (incoming_weights>cutoff)
        relevant_suppliers=incoming_weights.supplier[mask].data
        # node.children = 
        new_nodes.extend([Node(process_name=s) for s in relevant_suppliers])
        for _new in new_nodes:
            new_links.append((node.id,_new.id))
        # just the unique nodes
        # 
    return new_nodes, new_links

# still have a non-uniqueness issue but should be easier to remedy

transaction_table = np.array([[0.5,.5,0,0.4],
                  [0,0.1,0.9,0],
                  [0.3,0.4,0.3,0.1],
                  [0,0,0.2,0.8],
                  ])

sector_list = ['atoms','bits','cats','dogs']
direct_data = [1.8,0.4,0.2,0.1]
total_data = [2.4,0.8,0.6,0.5]
final_demand = [1,2,0.8,1.2]
# might not be legit numbers but just checking math

max_layers = 10


toy_chain = create_chain(transaction_table,final_demand,direct_data,total_data,sector_list,max_layers)

# apparently need a new class: Stage
# these are 1-1 with processes (just call it Process then) and populate a dict: {id:Process_instance} and {procname:instance}
# these are the nodes the graph cares about and get fed into the find_links function as the branch ends
# they are obtained from the new nodes list by finding the unique process names


root = Node(process_name='atoms')
layers = [[root]]
num_layers = 3
cutoff = 0.01
supply_chain = toy_chain
nodes = {root.id:root}

process_dict = {}
process_lookup = {}
for entity in supply_chain.user:
    name = entity.item()
    _p = Process(name=name)
    # will overwrite this so apply copies
    # _p_keep = copy(_p_temp)
    process_dict.update({_p.id:_p})
    process_lookup.update({name:_p})


In [180]:
# the oversampling issue with naive n-ary tree edge formation is a degeneracy
# it can be remedied by considering the hypergraph A(user,supplier,trophic_no) I think
# can do monte carlo experiments to look at surfacve area / volume ratio of the graphs with
# lognormally distributed terms. Spectral properties of these matrices? 
# Can we find a form for the distributions of the power series
# -> compute matrix norm and truncation information loss
# or simulate

# a_random = np.exp(





array([[ 538.57581053, 1261.57210621,  232.29826143,  566.71243022],
       [ 388.34090944, 1966.45661443,  476.22106697,  525.30436053],
       [ 249.98350383,  275.18855398, 1509.02012655,  188.27399099],
       [  80.98572902,   74.26217295,  332.15967248,  272.84745669]])

In [207]:

nmax = 20

num_nodes = 4
logmean = 6
logstd = 1
# add depletion term, the total output
loss = np.abs(2+0.8*np.random.randn(num_nodes,1))
seed = np.random.randn(num_nodes,num_nodes)


mean = 10
lognormal = np.exp(logstd*seed+logmean)
A = lognormal/(np.sum(lognormal,axis=1))-loss
traces = [np.trace(seed)]
for x in range(nmax):
    print(f"{np.trace(A)}")
    A = np.dot(A,traces[-1])
    traces.append(A)
px.line(y=traces).show()

-5.936336344678103
-6.462363974132204
33.46431385624446
1133.433193755369
1284772.5213274737
1650640440042.7996
2.7246138623046865e+24
7.423520698662862e+48
5.5108659563475944e+97
3.0369643588830887e+195
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf


In [ ]:

# still also need a lookup to find all the _nodes_ which visit a given process ... pah!

print(f'starting from {root.process_name}, {root.id}')
branch_ends = [root]

for layer in range(num_layers):
    weight_matrix = supply_chain.transfer.sel(layer=layer)
    print(f"There are {len(branch_ends)} leaves to extend at layer {layer}")
    new_nodes,new_links = find_links(weight_matrix,branch_ends,cutoff)
    # print('\n new_nodes:')
    print(new_nodes)
    # print('\n new_links:')
    # print(new_links)
    unique_processes = set([n.process_name for n in new_nodes])
    print(f"Found {len(new_links)} links to {len(new_nodes)} nodes ({len(unique_processes)} unique processes)")
    
    new_node_dict = {node.id:node for node in new_nodes}
    nodes.update(new_node_dict) # references to find all nodes by id
    # print('\nState of nodes:')
    # print([n.id for n in nodes.values()])
    # print([n.process_name for n in nodes.values()])
    layers.append(new_nodes) # unique nodes to check for next layer
    # print('\nState of layers:')
    # print(layers)
    
    # expand the tree
    for (old_id,new_id) in new_links:
        nodes[old_id].add_child(nodes[new_id]) # add the new edges
        
    # update the leading edge for the next iteration
    branch_ends = [process_lookup[p] for p in unique_processes]

In [123]:
new_links

[UUID('b67a6f3c-2d7e-417a-a98f-8d7e5ba8d181'),
 UUID('ddb40161-ca4d-47d8-bffd-9eb7cc13073a'),
 UUID('b67a6f3c-2d7e-417a-a98f-8d7e5ba8d181'),
 UUID('ec757ece-8ade-4a43-b242-93576081589b')]

In [46]:
from uuid import UUID


max_depth = 4
threshold = 1e-2 # proportion of total intensity below which to ignore
# the paths can definitely do with a better data structure 


# thresholds can be absolute or percentages of the total intensity of the target
# i.e. if the total intensity of a supplier is <X%, it's not important?
# could be made dynamic

supply_chain = toy_chain
root_name = 'atoms'
max_depth = 3
cutoff = 0.05


#todo implement this better method

# nodes = {root.id:root}
# layers = [[root]]

# def find_links(A,node_names,cutoff):
# as earlier implementation but
    # new_nodes = [Nodes_made_by_all_links]
    # new_links = [(id_old,id_new)]
    # return new_nodes, new_links

# for layer in range(num_layers):
    # branch_ends = layers[layer]
    # new_nodes,new_links = find_links(A,[node.name for node in branch_ends],cutoff)
    # nodes.update({node.id:node for node in new_nodes})
    # for (old_id,new_id) in new_links:
    # nodes[old_id].add_child(nodes[new_id])

def create_tree(supply_chain,root_name,max_depth,cutoff):# create the tree

        
    root = Node(root_name)
        
    num_layers = min(max_depth,len(supply_chain.layer))
    print(f'Scanning {num_layers} layers of a {len(supply_chain.layer)}-deep chain with cutoff {cutoff}')
        

    current_generation = [root] # empty list as this one has no parents
    surface_area = []
    total_nodes = 0

    # def add generation ...
    for layer in range(num_layers):
        # new nodes that are discovered in this pass
        # dict[name of process, [parents in this layer]]
        
        
        print(f'  -=-  Results at layer {layer}')
        transferred_intensity = supply_chain.absolute_direct*supply_chain.transfer.sel(layer=layer)
        # print(f'Transference:')
        # print(supply_chain.transfer.sel(layer=layer).data)
        # print(f'Intensities')
        # print(transferred_intensity.data)
        
        print(f"{len(current_generation)} parent nodes to check")
        print([s.process_name for s in current_generation])
        
        
        # beyond first layer, we come in with the list of nodes must be checke
        # and _then linked back to parents_ - which in the first instance is just root
        # 
        
        next_generation = []
        last_layer_dict: dict[UUID,Node] = {}
        heredity_dict: dict[str,list[UUID]] = {}
        for node in current_generation:
            # todo optimization: Don't do the calculation for a node that already exists
            # todo instead, just threshold the _processes_ and then attach the relevant
            # todo suppliers from each _process_ to the children of its _parents_
            
            
            # the keys (process) are the elements that need to be checked
            # when their contributions are determined, nodes lists
            # the parents that have to be updated
            this_layer_supply_intensity = transferred_intensity.sel(user=node.process_name)    
        
        
            supplier_mask = (this_layer_supply_intensity>cutoff)
            relevant_suppliers=this_layer_supply_intensity.supplier[supplier_mask].data
            node.children = [Node(process_name=s) for s in relevant_suppliers]
            next_generation.extend(node.children)
            # for s in relevant_suppliers:
                # if s not in heredity_dict.keys():
                    # heredity_dict[s] = [node.]
                # extend 
            print(' '*(1+layer) + f'{node.process_name} spawned {len(node.children)} children:')
            print(' '*(2+layer) +' '.join([n.process_name for n in node.children]))
        print('Next generation of processes:')# {process:spawning_parents}')
        print([p.process_name for p in next_generation])
            
        
        surface_area.append(len(next_generation))
        total_nodes+=len(next_generation)
        current_generation = set(next_generation) #just use names
        # update the parents
        
    print(f"Discovered {total_nodes} nodes in total\n")
    print('Nodes per layer')
    print(surface_area)

tree = create_tree(toy_chain,'atoms',3,0.05)
# comparison to pyspa
#  It took 14 seconds to extract 4746 pathways and calculate 4755 remainders.

Scanning 3 layers of a 10-deep chain with cutoff 0.05
  -=-  Results at layer 0
1 parent nodes to check
['atoms']
 atoms spawned 2 children:
  atoms cats
Next generation of processes:
['atoms', 'cats']
  -=-  Results at layer 1
2 parent nodes to check
['atoms', 'cats']
  atoms spawned 2 children:
   atoms bits
  cats spawned 3 children:
   atoms bits cats
Next generation of processes:
['atoms', 'bits', 'atoms', 'bits', 'cats']
  -=-  Results at layer 2
5 parent nodes to check
['atoms', 'bits', 'atoms', 'bits', 'cats']
   atoms spawned 3 children:
    atoms bits cats
   bits spawned 3 children:
    atoms bits cats
   atoms spawned 3 children:
    atoms bits cats
   bits spawned 3 children:
    atoms bits cats
   cats spawned 3 children:
    atoms bits cats
Next generation of processes:
['atoms', 'bits', 'cats', 'atoms', 'bits', 'cats', 'atoms', 'bits', 'cats', 'atoms', 'bits', 'cats', 'atoms', 'bits', 'cats']
Discovered 22 nodes in total

Nodes per layer
[2, 5, 15]


In [47]:
# check the structure of the tree
def show_tree(node):
    print(node.name)
    for child in node.children:
        print(child.name)
    # for child in node.children:
    #     show_tree(child)

show_tree(tree)

atoms
atoms
cats


In [ ]:
toy_chain = create_chain(transaction_table,final_demand,direct_data,total_data,sector_list,max_layers,from_A=True)
show_pld(toy_chain,'cats')

In [209]:
# use real data

tech_coef = np.genfromtxt('pyspa/A_matrix_template.csv',delimiter=',',skip_header=1)

# we can infer the (normalised) final demand from
#  inferred_demand = 1 - np.sum(tech_coef,axis=1)
# but this is only the proportion of output that serves demand, not an absolute number
# If using this then the scale of each sector is lost and should be included
# by using the absolute direct intensity per sector (ie in kgCO2e) instead of the 
# specific direct intensity (kgCO2e/$) in order to put absolute numbers on the PLD.
# Nonetheless, one can compute a _proportional_ PLD by setting the intensity vector to all 1s.

# extended_table = np.concatenate([transaction_table,np.atleast_2d(final_demand).T],axis=1)
# output = np.sum(extended_table,axis=1)
# tech_coef = transaction_table/np.atleast_2d(output).T

# sparsity about 1% ...

meta=pd.read_csv('pyspa/infosheet_template.csv')
# to retrieve nu
sector_list = list(meta['Name'])
sector_dict = meta[['Name','Sector number']].set_index('Name').to_dict()['Sector number']
# useful later
meta.set_index('Name',inplace=True)

satellite = meta.copy()
satellite.reset_index(inplace=True)
satellite['supplier'] = satellite['Name']
satellite = satellite[['supplier','DR_GHG_emissions_(kgCO2e)','TR_GHG_emissions_(kgCO2e)']]
satellite.rename(columns ={
    'DR_GHG_emissions_(kgCO2e)': 'direct_intensity',
    'TR_GHG_emissions_(kgCO2e)': 'total_intensity'
    } ,inplace=True)
# satellite.set_index(['supplier'],inplace=True)
# satellite_data = satellite.to_xarray()



# supply_chain_data = np.zeros((*tech_coef.shape,max_layers))
# layer_transfer = np.eye(tech_coef.shape[0])
# for layer in range(max_layers):
#     # create the next level of dependency
#     supply_chain_data[:,:,layer] = np.dot(layer_transfer,tech_coef)
#     layer_transfer = copy.copy(supply_chain_data[:,:,layer])

# chain = xr.Dataset(data_vars={'transfer':(['supplier','user','layer'],
#                                     supply_chain_data),
#                                     'direct_intensity':(['supplier'],direct_data),
#                                     'total_intensity':(['supplier'],total_data),
#                                     'final_demand':(['supplier'],final_demand),
#                                     'output':(['supplier'],output)}, 
#                         coords={'supplier':sector_list,
#                                 'user':sector_list,
#                                 'layer':range(max_layers)})



# # construct other useful objects for downstream use
# direct = chain.output*chain.direct_intensity
# direct.name = 'absolute_direct'
# chain=chain.merge(direct)

# total = chain.output*chain.total_intensity
# total.name = 'absolute_total'
# chain=chain.merge(total)



chain = create_chain(tech_coef,satellite.direct.values,
                     satellite.total.values,sector_list,max_layers=10)
chain

AttributeError: 'DataFrame' object has no attribute 'direct'

In [ ]:
# what about this guy?

In [48]:
big_tree = create_tree(chain,'Basic Chemical Manufacturing',10,0.01)

Scanning 3 layers of a 10-deep chain with cutoff 0.01
  -=-  Results at layer 0
1 parent nodes to check
['Basic Chemical Manufacturing']
 Basic Chemical Manufacturing spawned 6 children:
  Oil and gas extraction Petroleum and Coal Product Manufacturing Basic Chemical Manufacturing Electricity Generation Gas Supply Road Transport
Next generation of processes:
['Oil and gas extraction', 'Petroleum and Coal Product Manufacturing', 'Basic Chemical Manufacturing', 'Electricity Generation', 'Gas Supply', 'Road Transport']
  -=-  Results at layer 1
6 parent nodes to check
['Oil and gas extraction', 'Petroleum and Coal Product Manufacturing', 'Basic Chemical Manufacturing', 'Electricity Generation', 'Gas Supply', 'Road Transport']
  Oil and gas extraction spawned 1 children:
   Electricity Generation
  Petroleum and Coal Product Manufacturing spawned 2 children:
   Oil and gas extraction Electricity Generation
  Basic Chemical Manufacturing spawned 5 children:
   Sheep, Grains, Beef and Dairy 

In [ ]:
# flip the tree
stages = []
node = root


In [ ]:
# outputs from pyspa
# The percentage of contribution of that last node in that pathway, to the total intensity/multiplier/requirement of the selected sector/process is provided
# The value of the corresponding direct intensity/multiplier/requirement
# The value of the corresponding total intensity/multiplier/requirement
# The name of each node in the pathway, for each stage of the supply chain (1 to n).

In [ ]:
# their example uses threshold 1e-5, so let's adopt that and see if anything similar comes up


# PySPA

Rough workflow

SingleSUImportsAsPI()
    Prepares data for IO analysis in SU format

if there is a single region, there are imports
in a multi region (eg the tutorial), no imports 

SU_to_iot
    Optional 

IOanalysis
    Produces TIM and DIM (m and q)
    Works in IOT or SU
    If using SU for example, other vectors (x) need same extent (product,sector)

SPA


In [1]:

cd /workspaces/spa-path-exchange/

/workspaces/spa-path-exchange


/home/vscode/.cache/pypoetry/virtualenvs/non-package-mode-JBLCnCFX-py3.12/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# package is on pip but we'll probably want to mangle the source code
# that's actually pretty fast
#  It took 14 seconds to extract 4746 pathways and calculate 4755 remainders.
from pyspa import pyspa
sc = pyspa.get_spa(target_ID = 70, 
                   max_stage = 10,
                   a_matrix ='pyspa/A_matrix_template.csv', 
                   infosheet='pyspa/Infosheet_template.csv',
                   thresholds='pyspa/Thresholds_template.csv')

/workspaces/spa-path-exchange/pyspa/pyspa.py:51: SyntaxWarning: invalid escape sequence '\/'
  for char in "\/:*?<>|":


Reading infosheet...Done
Extracting names of satellites...Done
Reading Thresholds...Done
Reading A matrix...Done
Validating read data...The A matrix loaded is square and contains 114 sectors/processes across 1 region(s), and is described in the infosheet provided, for 1 satellite(s)...Done
Generating vectors of direct and total multipliers...Done
------ Ready to conduct the Structural Path Analysis ------
Supply Chain object created, extracting pathways, which will take some time...
Started at 01:46:43
Now calculating remainders
Ended at 01:46:57. It took 14 seconds to extract 4746 pathways and calculate 4755 remainders.
Started at 01:46:43


In [3]:
sc.export_to_csv('spa_results.csv')
# csv is broken, uneven number of entries per row
# Supply_Chain._generate_spa_table_header()  sets the number of 'stages'
# has attribute max_stages, use this to inform pathway.print_
# pathway.print_ makes a string which is as long as the num nodes
# so just need to add more tabs to make up the difference


# eg
# [Index reference 69 [S0], Index reference 45 [S1]]
# '2.923900%\t0.010636955136012228\t0.022528695752242738\tCement, Lime and Ready-Mixed Concrete Manufacturing\t'

#[Index reference 69 [S0], Index reference 36 [S1], Index reference 8 [S2]]
# '1.255238%\t0.004566472997184868\t0.006572335047809824\tPetroleum and Coal Product Manufacturing\tOil and gas extraction\t'
# seems like: num stages = num nodes - 1
# num_stage_delimiters = max_stage-1 # there isn't one at the end
# additional_tabs = max_stage-1 - num_nodes # each node appends a \t


In [ ]:
# 10 mins under the hood
# main function
# interface = Interface(A_matrix,infosheet,thresholds) # takes paths to files, not objs
# sc SupplyChain.make(interface,params)
# sc.extract_pathways()
# optionally calculate remainder
# save

# SupplyChain is the big monster


#init:
    # nothing too fancy
    # assign local attris
    # validate
    # add extraction stages
        # actually adds the method to the SupplyChain class, so in principle you could add another one
        # gets nonzero entries for a given sector and ??? creates next level of tree
        # '_extract_stage_%s_pathway is the method name
    
# extract_pathways
# # calls the extract_stage_1_pathway which then calls...
#                     if stage < self.max_stage:
#                         eval('self._extract_stage_%s_pathway(input_sector, acc_tech_coeff, thresholds_dict)' %
#                              str(stage + 1))
# that looks.... pretty sketchy! 
# it also does a whole lot of single-element slicing and multiplication, realistically this is probably where
# all the compute time goes... I think we can do much better with a matrix approach
# the Pathways are probably a useful construct thouhg

